In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
import query_processing.query_processing_spacy as qp

In [3]:
example = 'Have Apple stocks risen 40 million and what about Google stock?'

qp.get_ents(example)

['Apple', '40 million', 'Google']